In [1]:
import pdbufr
import sys
import traceback
 
from eccodes import *
from math import isnan

from ipywidgets import interact
import os
import birdy
import geopandas as gpd
import ipyleaflet
import ipywidgets

In [25]:
# Import bufr file of the TC track
df = pdbufr.read_bufr("../data/A_JSXX01ECEP250000_C_ECMP_20230725000000_tropical_cyclone_track_DOKSURI_124p6degE_17p6degN_bufr4.bin",
    columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
             "pressureReducedToMeanSeaLevel"))
# Save the number of ensembles in a list
members = df.ensembleMemberNumber.unique()
df.head()

,stormIdentifier,ensembleMemberNumber,latitude,longitude,pressureReducedToMeanSeaLevel
0,07W,1,17.7,124.5,96600.0
1,07W,1,18.5,123.9,96100.0
2,07W,1,19.0,122.8,96200.0
3,07W,1,19.4,121.7,95900.0
4,07W,1,19.3,121.0,95500.0


In [4]:
df2 = df[df.ensembleMemberNumber == 2]
df2_noNaN = df2.dropna()
print(df2_noNaN.latitude.iloc[0], df2_noNaN.longitude.iloc[0])

17.7 124.3


In [24]:
latitude = df2_noNaN.latitude.values
longitude = df2_noNaN.longitude.values
pressure = df2_noNaN.pressureReducedToMeanSeaLevel.values * 10**-2 # convert to hPa
location = [[latitude[i], longitude[i]] for i in range(len(latitude))]
print(pressure)

[ 963.  957.  958.  956.  955.  951.  948.  947.  937.  941.  951.  962.
  973.  979.  985.  990.  991.  993.  994.  995.  995.  996.  995.  994.
  998. 1001. 1003. 1001. 1002.]


In [11]:
# Save all ensemble members track in a list called locations
locations = []
for m in members:
    dft = df[df.ensembleMemberNumber == m]
    dft_noNaN = dft.dropna()
    lat = dft_noNaN.latitude.values
    lon = dft_noNaN.longitude.values
    locs = [[lat[i], lon[i]] for i in range(len(lat))]
    locations.append(locs)

In [41]:
# Build an interactive map with ipyleaflet

initial_lat_lon = (df2.latitude.iloc[0], df2.longitude.iloc[0])

tc_track_map = ipyleaflet.Map(
    center=initial_lat_lon,
    basemap=ipyleaflet.basemaps.OpenTopoMap,
)

# Add a custom zoom slider
zoom_slider = ipywidgets.IntSlider(description="Zoom level:", min=1, max=17, value=4)
ipywidgets.jslink((zoom_slider, "value"), (tc_track_map, "zoom"))
widget_control1 = ipyleaflet.WidgetControl(widget=zoom_slider, position="topright")
tc_track_map.add_control(widget_control1)

# Add tropical cyclone track to the map
track1 = ipyleaflet.Polyline(
    locations=locations[0],
    color="green" ,
    fill=False,
    name='ENS 1',
)
tc_track_map.add_layer(track1)

# Add tropical cyclone track to the map
track2 = ipyleaflet.Polyline(
    locations=locations[1],
    color="red" ,
    fill=False,
    name='ENS 2',
)
tc_track_map.add_layer(track2)

# Add layers widget to the map
tc_track_map.add_control(ipyleaflet.LayersControl());

# Add a marker to the map
# marker = ipyleaflet.Marker(location=initial_lat_lon, draggable=True)
# tc_track_map.add_layer(marker)

In [42]:
tc_track_map

Map(center=[17.7, 124.3], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…